Do the above scraping for multiple matches

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-notifications")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(5)

base_url = 'https://www.cricbuzz.com'
archives_url = f'{base_url}/cricket-scorecard-archives'
driver.get(archives_url)
time.sleep(3)  

soup = BeautifulSoup(driver.page_source, 'html.parser')
match_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if '/live-cricket-scorecard/' in href:
        full_url = base_url + href
        if full_url not in match_links:
            match_links.append(full_url)

match_links = match_links[:5]

all_matches_data = []
for link in match_links:
    driver.get(link)
    time.sleep(2)  
    match_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    title_tag = match_soup.find('h1', class_='cb-nav-hdr cb-font-18 line-ht24')
    match_title = title_tag.text.strip() if title_tag else 'N/A'
    
    team_tags = match_soup.find_all('a', class_='cb-nav-tab')
    teams = [tag.text.strip() for tag in team_tags] if team_tags else ['N/A', 'N/A']
    
    score_tags = match_soup.find_all('div', class_='cb-col cb-col-100 cb-scrd-hdr-rw')
    scores = [tag.text.strip() for tag in score_tags] if score_tags else ['N/A']
    
    match_data = {
        'Match Title': match_title,
        'Teams': teams,
        'Scores': scores
    }
    all_matches_data.append(match_data)

df = pd.DataFrame(all_matches_data)
print(df)

driver.quit()

Empty DataFrame
Columns: []
Index: []
